# Install and Import Dependencies

In [7]:
# install modules stable-baselines3 includes gym
!pip install stable-baselines3[extra] 

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616820 sha256=33bdd11219aeaca35fb0e9c841a70715e9e62cf49222d6f38557ec2f53cb67c4
  Stored in directory: c:\users\jake\appdata\local\pip\cache\wheels\b3\50\6c\0a82c1358b4da2dbd9c1bb17e0f89467db32812ab236dbf6d5
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=446464 sha256=a9057a4a2dc611f217f05e70bc22372f3edc45c5af19b7f74a17d36eb16ef904
  Stored in directory: c:\users\jake\appdata\local\pip\cache\wheels\2b\03\e4\8b662e95b85786a03898fca125d5a9e3fe49337b1eba8fddd2
Successfully built gym AutoROM.accept-rom-license
  Attempting uninstall: importlib-metadata
 

In [9]:
# !pip install tensorflow gym keras keras-rl2

In [42]:
# DQN = Deep-Q-Network - maximize bellman equation, MLP Policy for model
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Tuple
import numpy as np
import os
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env

# Building the Environment

In [46]:
# class for maze env where agent will learn
class MazeEnv(Env):
    
    # maze member_variables & action / observation space
    def __init__(self, DIM_COL_ROW, STARTING_CELL):
        # declare num_col & num_row; boards are square
        self.dim_row_col = DIM_COL_ROW
        # declare starting cell
        self.starting_cell = STARTING_CELL
        # actions we can take, up, right, down, left ; NESW
        self.action_space = Discrete(4) # 0,1,2,3
        # observation 
        # option1 - discrete(36) w/ % to make 6x6 grid
        # option2 - Tuple((Discrete(DIM_COL_ROW), Discrete(DIM_COL_ROW))
        # taking option 2 bc tuple exists; vscode says stable not supported from check_env, but jupyter no error, curious
        self.observation_space = Tuple((Discrete(DIM_COL_ROW), Discrete(DIM_COL_ROW)))
        # set starting cell - note self.state updates per step
        self.state = STARTING_CELL
        # set max_steps to prevent infinite searching in maze
        self.max_steps = 1000 # 1000 baseline, will change
        # set current step for iterating action steps
        self.current_step = 0
        # set value for cell in maze that ends episode
        self.end_cell = (DIM_COL_ROW,DIM_COL_ROW)
        # set episode termination variable to false
        self.episode_terminated = False
        
    # note - impossible move below pass increments; optimize by logic on step
    # how about step incrementer in else (non-pass steps)
        
    # moves agent around env; how actions change states
    def step(self, action):
        # action discrete values defined below
        # 0 is down 
        # 1 is left
        # 2 is up
        # 3 is right
        
        # take action & change state cell; passes prevent action in else
        if (action == 0 or action == 2):
            # pass if at bottom and go down
            if (self.state[0] == 0 and action == 0):
                pass
            # pass if at top and go up
            elif (self.state[0] == self.dim_row_col and action == 2):
                pass
            # take action up or down when not on border
            else:
                # 2 - 1 => go up 1; 0 - 1 => go down 1 
                self.state = (self.state[0] + (action - 1), self.state[1])
        # else action = 1 or 3
        else:
            # pass if at left and go left
            if (self.state[1] == 0 and action == 1):
                pass
            # pass if at right and go right
            elif (self.state[1] == self.dim_row_col and action == 3):
                pass
            # take action up or down when not on border
            else:
                # 3 - 2 => go right 1; 1 - 2 => go left 1 
                self.state = (self.state[0], self.state[1] + (action -2))
                
        # increment max_steps as action was taken - note pass above
        self.current_step += 1
        
        # calculate reward & check if at end-condition
        if (self.state == self.end_cell):
            self.episode_terminated = True
            reward = 100
        else:
            # incentive to keep moving; reach end quickly
            reward = -1/(self.dim_row_col*self.dim_row_col)
            
        # end-condition w/ out reward is too many steps taken
        if (self.current_step >= self.max_steps):
            self.episode_terminated = True
            
        # info {} must be returned for step() for Env class; dk why
        info = {}
        
        # return step information
        return self.state, reward, self.episode_terminated, info
                
    # implements visuals of learning process
    # COULD USE PYGAME ??? - See mattchan maze_view_2d.py
    def render(self):
        # implement viz
        pass
    
    # reset sets env's params to starting values
    def reset(self):
        # reset state to starting cell
        self.state = self.starting_cell
        # reset episode_terminated to episode running
        self.episode_terminated = False
        # reset current step to no steps taken
        self.current_step = 0
        # return state to exploit; model.predict(env.reset()) in test model
        return self.state
        

In [47]:
# declare environment
DIM_COL_ROW = 6
STARTING_CELL = (0,0)
env = MazeEnv(DIM_COL_ROW, STARTING_CELL)
# ensure env functions with stable_baselines well
check_env(env, warn=True)

# Test Environment

In [54]:
episodes = 5
# iterate through simulated episodes of the env with random actions
for episode in range(1, episodes+1):
    # make state at starting cell
    state = env.reset()
    # reset boolean for simulation
    done = False
    # score is metric for rewards
    score = 0 
    
    # simulate a singular episode with random action
    while not done:
        # render env
        env.render()
        # random actions hence .sample()
        action = env.action_space.sample()
        # step from state with action & capture step info
        n_state, reward, done, info = env.step(action)
        # increment reward metric
        score+=reward
    # display simulation results
    print('Episode:{} Score:{} Steps:{}'.format(episode, score, int((100-score)*(DIM_COL_ROW*DIM_COL_ROW))))
# env.close() - needed when render implemented

Episode:1 Score:98.19444444444444 Steps:65
Episode:2 Score:98.38888888888889 Steps:58
Episode:3 Score:92.91666666666669 Steps:254
Episode:4 Score:86.44444444444431 Steps:488
Episode:5 Score:97.69444444444444 Steps:83
